# Импорт библиотек и загрузка данных

In [13]:
import json
import cv2 
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import seaborn as sns
import pandas as pd
from collections import Counter
from functools import lru_cache
import os
import shutil
import re
import logging
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms, models
from torch import nn
from torch import optim
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import torch
import timm

c:\Users\maksi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Считываем данные

In [2]:
class ImageDataset:
    def __init__(self, path, path2image, n_classes=15):
        # Загрузка данных из CSV
        self.df = pd.read_csv(path)
        self.path2image = Path(path2image)
        self.N = n_classes
        
    def __len__(self):
        return len(self.df)

    def _encode_label(self, classes):
        label = np.zeros(self.N)
        for c in np.array(classes, dtype=np.int16) - 1:
            label[c] = 1
        return label
        
    def sample(self, clss=1, count=15):
        files = []
        while len(files) < count:
            indx = np.random.randint(0, len(self.df))
            labels = self.df.iloc[indx]["OUTPUT:classes"]
            try:
                if clss in self.parse(labels):
                    files.append(self.path2image / self.file_name(indx))
            except:
                continue
        return files
    
    def create_path(self, indx):
        return str(self.path2image / self.file_name(indx))
        
    def __getitem__(self, indx):
        img = cv2.imread(self.create_path(indx))
        clases = self.parse(self.df.iloc[indx]["OUTPUT:classes"])
        label = self._encode_label(clases)
        sample = {'image': img,
                  'label': label}
        return sample
            
    def file_name(self, indx):
        return self.df.iloc[indx]['file_name']
    
    def resize(self, img, scale=2):    
        shape = (np.array(img.shape[:2]) / scale).astype(np.int32)
        return cv2.resize(img, shape)
    
    def parse(self, st):        
        # Преобразуем строку вида '[2, 5, 11]' в список [2, 5, 11]
        return [int(x) for x in re.findall("\d+", st)]

<>:49: SyntaxWarning: invalid escape sequence '\d'
<>:49: SyntaxWarning: invalid escape sequence '\d'
C:\Users\maksi\AppData\Local\Temp\ipykernel_10844\1111695564.py:49: SyntaxWarning: invalid escape sequence '\d'
  return [int(x) for x in re.findall("\d+", st)]


In [3]:
class TorchImageDataset(ImageDataset, Dataset):
    def __init__(self, path, path2image, imgsz=256, MEAN = (0.485, 0.456, 0.406), STD = (0.229, 0.224, 0.225)):
        super().__init__(path, path2image)
        self.imgsz = imgsz
        self.fransform_img= transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((self.imgsz, self.imgsz)),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomRotation(10),    
                    transforms.ColorJitter(brightness=0.2, 
                                                   contrast=0.2, 
                                                   saturation=0.2, 
                                                   hue=0.2),
                    transforms.Normalize(mean=MEAN, std=STD),
        ])
            
    @lru_cache(10000)
    def __getitem__(self,indx):
        img, label = super().__getitem__(indx).values()
        if img is not None:
            img = self.fransform_img(img)
            return {'image':img,'label':label}

## Подготовка файла

## Получаем датасет

In [4]:
dataset = TorchImageDataset("./analis/data.csv",  "../Resize/")

## DataLoader

In [5]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():  
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device       

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            img,label = tuple(b.values())
            img = to_device(img,self.device)
            label = to_device(label, self.device)
            yield {"image":img,"label":label}

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [6]:
def split_data(dataset, train_ratio=0.7, val_ratio=0.1):

    total_size = len(dataset)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)
    test_size = total_size - train_size - val_size

    train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

    return train_data, val_data, test_data

In [7]:
train_data, val_data, test_data = split_data(dataset, train_ratio=0.6, val_ratio=0.05)

In [55]:
type(dataset[0])

dict

In [9]:
device = torch.device("cuda:0")

In [10]:
len(dataset)

14594

In [11]:
train_loader = DeviceDataLoader(DataLoader(train_data, batch_size=16,num_workers = 0), device=device)
val_loader   = DeviceDataLoader(DataLoader(val_data, batch_size=16,num_workers = 0), device=device)
test_loader   = DeviceDataLoader(DataLoader(test_data, batch_size=16,num_workers = 0), device=device)

# Обучение модели

In [14]:
class ModelMultilabel(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes        
        self.model = None
        self.model = self.__efficientnet_b5()
        
    def __efficientnet_b5(self):
        model = models.efficientnet_b5(pretrained=True)
        model.classifier[1] = nn.Linear(2048,self.n_classes)
        return model
    def forward(self, input):
        out = self.model(input)
        out = F.sigmoid(out)
        return out

def train_model(model, dataloader_train, dataloader_valid, learningRate, num_epochs, device = 'cuda:0'):
    logger = logging.getLogger('api_log')
    logger.setLevel(logging.INFO)
    file_handler = logging.FileHandler(f'log_{learningRate}.txt')
    file_handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.debug(f"Start_train  lr={learningRate}")
    
    # Установка оптимизатора с текущей скоростью обучения
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
    criterion = nn.CrossEntropyLoss()
    # Цикл обучения
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_labels = []
        all_predictions = []

        for s in tqdm(dataloader_train, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
            if s is None:
                continue
            inputs = s['image']
            labels = s['label']
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)  # Выходы модели
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            predicted = outputs
            all_labels.extend(labels.detach().cpu().numpy())
            all_predictions.extend(predicted.detach().cpu().numpy())
        # Средняя потеря за эпоху
        epoch_loss = running_loss / len(dataloader_train)
        # precision = precision_score(all_labels, all_predictions, average='weighted')
        # recall = recall_score(all_labels, all_predictions, average='weighted')
        # accuracy = accuracy_score(all_labels, all_predictions)
        f_score = F_score(all_predictions, all_labels)
        # print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, "
        #       f"Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}, F_score: {f_score:.4f}")
        logger.info(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, F_score: {f_score:.4f}")
        
        # logger.debug(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f},  F_score: {f_score:.4f}")
        validate_model(model, dataloader_valid, logger)    
    # Сохранение модели
    torch.save(model.state_dict(), f'model_{learningRate}.pth')
    
def validate_model(model, dataloader_valid, logger, device='cuda:0'):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for s in tqdm(dataloader_valid, desc='Validation'):
            if s is None:
                continue
            inputs = s['image']
            labels = s['label']
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            predicted = outputs
            all_labels.extend(labels.detach().cpu().numpy())
            all_predictions.extend(predicted.detach().cpu().numpy())

    # precision = precision_score(all_labels, all_predictions, average='weighted')
    # recall = recall_score(all_labels, all_predictions, average='weighted')
    # accuracy = accuracy_score(all_labels, all_predictions)
    f_score = F_score(all_predictions, all_labels)
    # print(f"Validation Results - Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}")
    # logger.debug(f"Validation Results - Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}, F_score: {f_score:.4f}")
    logger.info(f"Validation Results -  F_score: {f_score:.4f}")


def F_score(output, label, threshold=0.5, beta=1): #Calculate the accuracy of the model
    if isinstance(output, list):
        output = np.array(output)
    if isinstance(label, list):
        label = np.array(label)
    print(label.shape, output.shape)
    assert (output.shape == label.shape),'shape is different'
    prob = output > threshold
    label = label > threshold

    TP = (prob == label).sum()
    TN = (np.bitwise_not(prob) == np.bitwise_not(label)).sum()
    FP = (prob == np.bitwise_not(label)).sum()
    FN = (np.bitwise_not(prob) == label).sum()
    print(TP, TN, FP, FN)
    precision = np.mean(TP / (TP + FP + 1e-12))
    recall = np.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2


In [15]:
model = ModelMultilabel(15).to(device)

c:\Users\maksi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\maksi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B5_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [70]:
dataset[1]['label']

array([0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.])

In [16]:
train_model(model, train_loader, val_loader, 0.001, 50)

Training Epoch 1/50: 100%|██████████| 548/548 [25:15<00:00,  2.76s/it]


(8756, 15) (8756, 15)
111503 111503 19837 19837


Validation: 100%|██████████| 46/46 [01:53<00:00,  2.47s/it]


(729, 15) (729, 15)
9292 9292 1643 1643


Training Epoch 2/50: 100%|██████████| 548/548 [03:33<00:00,  2.57it/s]


...

(729, 15) (729, 15)
9993 9993 942 942


Training Epoch 50/50: 100%|██████████| 548/548 [03:18<00:00,  2.77it/s]


(8756, 15) (8756, 15)
125670 125670 5670 5670


Validation: 100%|██████████| 46/46 [00:04<00:00, 10.38it/s]


(729, 15) (729, 15)
9947 9947 988 988


In [78]:
torch.save(model, "model80.pth")

In [ ]:
model = torch.load("model.pth")
model.eval()

## Метрики

In [17]:
model.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        inputs = batch['image'].to(device)
        labels = batch['label'].to(device)

        outputs = model(inputs)
        outputs = (outputs > 0.5).float()

        all_predictions.append(outputs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())
    
all_predictions = np.vstack(all_predictions)
all_labels = np.vstack(all_labels)
    
f1 = f1_score(all_labels, all_predictions, average='weighted')  
print(f"F1-Score: {f1:.4f}")

Evaluating: 100%|██████████| 320/320 [13:22<00:00,  2.51s/it]

F1-Score: 0.7977


In [19]:
f1_score(all_labels, all_predictions, average='macro')  

np.float64(0.5278923969003797)

In [20]:
f1_score(all_labels, all_predictions, average='micro')  

np.float64(0.8147874516935667)

In [21]:
import numpy as np
from sklearn.metrics import f1_score
import torch
from tqdm import tqdm

# Переводим модель в режим оценки
model.eval()

all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        inputs = batch['image'].to(device)
        labels = batch['label'].to(device)

        outputs = model(inputs)
        outputs = (outputs > 0.5).float()  # Если задача бинарной классификации, иначе можно использовать softmax

        all_predictions.append(outputs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Преобразуем списки в массивы
all_predictions = np.vstack(all_predictions)
all_labels = np.vstack(all_labels)

# Вычисляем F1-Score для каждого класса
f1_per_class = f1_score(all_labels, all_predictions, average=None)

# Выводим F1-Score для каждого класса
for i, f1 in enumerate(f1_per_class):
    print(f"Class {i + 1} F1-Score: {f1:.4f}")

Evaluating: 100%|██████████| 320/320 [00:33<00:00,  9.50it/s]

Class 1 F1-Score: 0.7377
Class 2 F1-Score: 0.8680
Class 3 F1-Score: 0.4062
Class 4 F1-Score: 0.4911
Class 5 F1-Score: 0.9222
Class 6 F1-Score: 0.0000
Class 7 F1-Score: 0.8441
Class 8 F1-Score: 0.8781
Class 9 F1-Score: 0.4490
Class 10 F1-Score: 0.8088
Class 11 F1-Score: 0.8221
Class 12 F1-Score: 0.0000
Class 13 F1-Score: 0.6909
Class 14 F1-Score: 0.0000
Class 15 F1-Score: 0.0000
